In [1]:
import os
import numpy as np
import pandas as pd

DATA_DIR = "/data/tmp/arogya/data_dec"
YEAR = 2019

In [2]:
ids = os.listdir(f"{DATA_DIR}/dnq_outputs")
ids

['1606_0',
 '2303_0',
 '2303_1',
 '1702_0',
 '1702_1',
 '1702_2',
 '3103_0',
 '3103_1',
 '3103_2',
 '3103_3',
 '3103_4',
 '808_0',
 '2505_0',
 '1903_0',
 '1903_1',
 '1903_2',
 '1903_3',
 '2102_0',
 '2102_1',
 '2102_2',
 '1112_0',
 '1112_1',
 '2401_0',
 '1807_0',
 '1807_1',
 '2006_0',
 '2006_1',
 '2006_2',
 '2006_3',
 '1124_0',
 '1124_1',
 '1124_2',
 '804_0',
 '605_0',
 '605_1',
 '605_2',
 '805_0',
 '901_0',
 '901_1',
 '1125_0',
 '2709_0',
 '2709_1',
 '813_0',
 '1113_0',
 '1113_1',
 '2416_0',
 '2103_0',
 '2103_1',
 '2103_2',
 '2103_3',
 '2103_4',
 '2103_5',
 '1902_0',
 '1902_1',
 '1001_0',
 '3102_0',
 '3102_1',
 '809_0',
 '1703_0',
 '2601_0',
 '2601_1',
 '2601_2',
 '2302_0',
 '2302_1',
 '1312_0',
 '1607_0',
 '2705_0',
 '2705_1',
 '2705_2',
 '2705_3',
 '3401_0',
 '814_0',
 '2309_0',
 '2309_1',
 '2309_2',
 '802_0',
 '1708_0',
 '1708_1',
 '1118_0',
 '2108_0',
 '2108_1',
 '2108_2',
 '2702_0',
 '2702_1',
 '2702_2',
 '2702_3',
 '1315_0',
 '2305_0',
 '2305_1',
 '2305_2',
 '2313_0',
 '2313_1',


In [3]:
len(np.unique([i.split("_")[0] for i in ids ]))

146

In [4]:
clustering_results_tile = None
for id_ in ids:
    if "outputs" in os.listdir(f'{DATA_DIR}/dnq_outputs/{id_}'):
        data = pd.read_csv(f'{DATA_DIR}/dnq_outputs/{id_}/outputs/predictions/kmeans_3_diff_bands/kmeans_3_diff_bands.csv')
        data['distid'] = id_.split("_")[0]
        data['tileid'] = id_.split("_")[1]
        if clustering_results_tile is None:
            clustering_results_tile = data
        else:
            clustering_results_tile = pd.concat([clustering_results_tile, data], axis=0)
#         print(clustering_results_tile.tail(2))
        
        #         print(os.listdir(f'{DATA_DIR}/dnq_outputs/{id_}/outputs/predictions/kmeans_3_diff_bands/kmeans_3_diff_bands.csv'))

In [5]:
clustering_results = clustering_results_tile.groupby(['distid', 'clust']).sum().reset_index()
clustering_results['distid'] = clustering_results['distid'].astype('int64')

In [48]:
zero=clustering_results[clustering_results['clust']==0]
one=clustering_results[clustering_results['clust']==1]

In [49]:
poppy_counts = pd.read_csv("/data/tmp/arogya/data_dec/inputs/poppy_1994-2020.csv")
# poppy_counts['distid'] = poppy_counts['distid'].astype(str)

In [50]:
poppy_counts = poppy_counts[['distid', f'X{YEAR}']]


In [52]:
type(zero)
one_j = pd.merge(one, poppy_counts)
one_j['quintile'] = pd.qcut(one_j[f'X{YEAR}'], 4, labels=False)
zero_j = pd.merge(zero, poppy_counts)
zero_j['quintile'] = pd.qcut(zero_j[f'X{YEAR}'], 4, labels=False)




# pd.merge([zero, poppy_counts],how='inner', on='distid', right=0)
# clustering_results.join(one, how='inner', on='distid')

In [53]:
zero_j.groupby('quintile')[['clustering_ha', f'X{YEAR}']].corr()

clustering_ha     X2019
quintile                                       
0        clustering_ha       1.000000 -0.123974
         X2019              -0.123974  1.000000
1        clustering_ha       1.000000 -0.161130
         X2019              -0.161130  1.000000
2        clustering_ha       1.000000  0.393932
         X2019               0.393932  1.000000
3        clustering_ha       1.000000  0.494057
         X2019               0.494057  1.000000

In [66]:
from IPython.display import display, HTML
corrs = one_j.groupby('quintile')[['clustering_ha', f'X{YEAR}']].corr().reset_index()
# corrs[corrs['']]
display(corrs[corrs['level_1']=='clustering_ha'][['quintile', f'X{YEAR}']].rename({f'X{YEAR}': 'pearson_correlation'},axis=1))

,quintile,pearson_correlation
0,0,-0.116650
2,1,0.116561
4,2,0.301997
6,3,0.811612


In [68]:
pd.set_option('display.max_rows', 200)
display(one_j.rename({'clustering_ha':'ground_truth_ha', f'X{YEAR}': 'predicted_ha', 'quintile':'quintile_ground_truth'}, axis=1).drop(['clust', 'pixels_from_clustering' ], axis=1))

,distid,ground_truth_ha,predicted_ha,quintile_ground_truth
0,1001,138.88,57.0,1
1,1002,168.44,49.0,1
2,1003,19.77,62.0,1
3,1004,44.45,62.0,1
4,1005,46.40,11.0,0
5,1006,45.50,29.0,0
6,1007,120.16,85.0,1
7,1009,27.90,1.0,0
8,1010,212.54,24.0,0
9,1011,1252.91,1.0,0
